In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/trainX.npy')
y_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/trainy.npy')
X_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testX.npy')
y_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [ ]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [ ]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=10, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 119, 64)           1984      
                                                                 
 conv1d_3 (Conv1D)           (None, 110, 64)           41024     
                                                                 
 dropout_1 (Dropout)         (None, 110, 64)           0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 55, 64)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3520)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               352100    
                                                      

In [ ]:
history = model.fit(X_training, y_training, epochs=10, verbose=False, validation_data=(X_validation, y_validation), batch_size=100)

In [ ]:
loss, accuracy = model.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9509
Testing Accuracy: 0.8829


In [ ]:
y_pred = model.predict(X_test, verbose=False)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)

In [48]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred )
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred, average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred, average='weighted')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred, average='macro')
print('F1 score: %f' % f1)

Accuracy: 0.882888
Precision: 0.882888
Recall: 0.882888
F1 score: 0.856812


In [ ]:
# ROC AUC
auc = model.compile('sgd', loss='mse', metrics=[tf.keras.metrics.AUC()])
print(auc)

None
